<a href="https://colab.research.google.com/github/kimtoto23/LGAI_23_2/blob/main/lgai_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Setting GPU & import Package


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 그래프 한글 폰트 깨짐 해결용

# 이거 한번 실행하고, 런타임 재시작 꼭!!!! 하고 아래에 폰트 지정 코드 실행

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('./data')

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams["figure.figsize"] = (12,5) # 매번 figsize로 그래프 크기 지정하기 귀찮으니까 default 크기를 지정

import seaborn as sns
from datetime import timedelta, date
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
from prophet import Prophet
from scipy import signal
from scipy import stats

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# from parallel import DataParallelModel, DataParallelCriterion
import optuna
import sklearn
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import joblib
from optuna.trial import TrialState

In [ ]:
torch.cuda.is_available()

True

In [ ]:
print(torch.cuda.device_count())

2


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 2. 원래 raw train은 어떻게 생겼는가

In [ ]:
train = pd.read_csv('train.csv')
train.groupby('대분류').sum().drop(columns = 'ID').transpose().plot()
# 그래프를 확인해보면, 2023-02-05 이후에 굉장히 부자연스럽게 판매량이 뚝 감소하는 것을 확인

In [ ]:
plt.scatter((train.loc[:,'2023-02-23':'2023-03-28'] != 0).sum(1).index, (train.loc[:,'2023-02-23':'2023-03-28'] != 0).sum(1).values, s = 10, alpha = 0.9)
plt.title('2023-02-23 부터 2023-03-28 기간 사이에 판매량이 0이 아닌 점이 얼마나 있는지')
plt.show()
# 15890개 제품 중, '2023-02-23':'2023-03-28' 기간 동안 판매량이 0이 아닌, 실제 판매가 발생한 기간이 얼마인지 산점도로 표현
# 실제로 1이상에 타점된 부분은 전체중 일부분에 불과한 것을 확인.

In [ ]:
((train.loc[:,'2023-02-23':'2023-03-28'] != 0).sum(1) == 0).sum() # 이건 해당 기간동안 판매가 단 하나도 안된 애들이 몇개인지 표현해준다.
# 총 14722개 제품이 해당 기간동안 제품이 하나도 팔리지않음.
# 이건 무언가 잘못된 정보라고 판단하여 해당 기간동안 값을 결측으로 판단, 보간을 실시한다.

In [ ]:
zero = train.iloc[((train.loc[:,'2023-02-23':'2023-03-28'] != 0).sum(1) == 0).values,:] # 이건 전체 train 중에서 02/23~03/28 에 집계가 안된 것들
not_zero = train.iloc[((train.loc[:,'2023-02-23':'2023-03-28'] != 0).sum(1) != 0).values,:] # 이건 전체 train 중에서 02/23~03/28 에 집계가 제대로 된 것들
print(zero.shape, not_zero.shape, zero.shape[0] + not_zero.shape[0], '정확하게 나눠진 모습')
# zero변수에는 결측이 있는 제품들, not_zero에는 결측이 없이 잘 집계가 된 제품을 모아둠.

# 추후에 zero변수에 대해서만 보간을 실시하고 pd.concat()을 통해 행으로 합친 후, ID 기준으로 sort_values()할 계획

In [ ]:
zero.groupby('대분류').sum().drop(columns = 'ID').transpose().plot();
plt.title('판매량이 0인 것들만 모아놓은 train')
not_zero.groupby('대분류').sum().drop(columns = 'ID').transpose().plot();
plt.title('판매량이 0이 아닌 것들만 모아놓은 train')

# 그래프를 따로 그리니 더욱 확실하게 이상한 점이 보인다.

## raw Data 에서 일자별 boxplot 을 그려보자

- 이상한 점을 더욱 자세히 확인해보기 위해 raw train 에서 일자별 제품들의 boxplot 을 그려보자


In [ ]:
df = pd.read_csv('./train.csv')
df.head()

In [ ]:
## 컬럼들만 따로 빼오자
columns_to_plot = df.columns[6:-1]
columns_to_plot

In [ ]:
data_melted = pd.melt(df,  value_vars=columns_to_plot, var_name='column')
data_melted
# 그래프 그리기
plt.figure(figsize=(120, 6))  # 그래프의 크기 설정
sns.boxplot(x='column', y='value', data=data_melted)  # x에는 컬럼명, y에는 값을 지정합니다.
plt.title("Boxplots of Columns")  # 그래프 제목 설정
plt.xticks(rotation=90)  # x 축 레이블 회전
plt.show()  # 그래프 출력

- 2023-02-23 ~ 2023-03-28 기간에 판매량이 현저히 적은 것을 확인할 수 있다,
- 해당 구간을 포함한 일정 구간의 box plot 을 확인해보자

In [ ]:
columns_to_plot = df.columns[-90:-1]
columns_to_plot

In [ ]:
data_melted = pd.melt(df,  value_vars=columns_to_plot, var_name='column')
data_melted
# 그래프 그리기
plt.figure(figsize=(20, 6))  # 그래프의 크기 설정
sns.boxplot(x='column', y='value', data=data_melted)  # x에는 컬럼명, y에는 값을 지정합니다.
plt.title("Boxplots of Colnmns")  # 그래프 제목 설정
plt.xticks(rotation=90)  # x 축 레이블 회전
plt.show()  # 그래프 출력

- 확실히 판매량이 이상치라고 할 수 있을만큼 현저히 감소하는 것을 볼 수 있다.
- 추후 이 기간의 판매량을 대체할 수 있는 만한 보간법을 생각해야 할 것 같다.

### EXTRA:  제품군 분류를 확인해보자

- 본격적인 분석에 앞서 대분류/중분류/소분류별 속한 그룹들을 확인해보자

In [ ]:
train_data = pd.read_csv('./train.csv')

In [ ]:
# 대,중,소 로 묶어서 개수를 확인해보자
result = train_data.groupby(['대분류', '중분류', '소분류']).size()
result

In [ ]:
# 브랜드까지 추가해서 확인해보자
pd.set_option('display.max_rows', None)
result = train_data.groupby(['대분류', '중분류', '소분류', '브랜드']).size()
# 출력 너무 많이 나와서 조금만 확인
result.head(20)

In [ ]:
## 위 분류를 기준으로 대략적으로 데이터 프레임의 구성을 확인해보자
train_data_3 = train_data.groupby(['대분류', '중분류', '소분류','브랜드']).agg(lambda x: x)
train_data_3.head(20)

- 특정 데이터들이 리스트로 구성되어 있다.
- 해당 값이 여러개 입력되어 있는 것은 동일한 대,중,소,브랜드 가 존재해서 그런듯 하다.
- 특정 분류값을 대상으로 이를 확인해보자.

In [ ]:
desired_value = ('B002-C001-0001', 'B002-C002-0001', 'B002-C003-0001', 'B002-00043')
result = train_data[(train_data['대분류'] == desired_value[0]) & (train_data['중분류'] == desired_value[1]) & (train_data['소분류'] == desired_value[2])& (train_data['브랜드'] == desired_value[3])]
result

- 분류에도 중복된 값이 있는 것을 확인

* 판매량이 잘 집계된 not_zero는 그대로 가져가고, 집계가 안된 zero들에 대해서만 예측으로 보간을 실시

# 3. Prophet으로 보간하기
* 후반부에 0인 것들만 골라서 보간 (14772개)
* 0인 것들의 2023년 2월 22일 까지 데이터로 2월23일 ~ 3월 28일 제품 판매량을 보충 이후 3월 29일부터 4월 4일까지는 원래 값 그대로 사용.

## 0인 것 전체 평균

## prophet 사전준비

In [ ]:
data = zero.loc[:,'2022-01-01':'2023-02-22'].mean() # 우리가 예측을 시작할 2023-02-23 직전까지 판매량은 전부 사용.
# 우리는 기준을 평균으로 잡았다. 각 날짜의 평균 판매량을 알고, 그것으로 하이퍼 파라미터를 조정할 수 있다고 생각.
series = data.reset_index()
series.columns = ['ds','y']
series.head(3)
# 이런 형태는 Prophet 모델에서 요구하는 형태를 맞추기 위해서.

In [ ]:
series.plot();
# 각 날짜의 평균판매량으로 그래프를 그려보았다.

## prophet 적용

### 월계절성 추가, 트렌드 유연성 증가, 분산 가변형(multiplicative) 확인, KR 휴일 추가

* 이는 이미 시행착오 끝에 지정한 하이퍼 파라미터입니다.

In [ ]:
m = Prophet(
    # trend
    changepoint_prior_scale=0.5, # 경향을 더 유연하게 받아들임.
    # seasonality
    seasonality_prior_scale = 0.5, # 계절성도 더 유연하게 받아들임.
    weekly_seasonality = 20,
    daily_seasonality=False,
    yearly_seasonality=False,
    seasonality_mode='multiplicative' # 계절성의 크기를 가변형으로 잡고
)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5) # monthly 계절성 추가
m.add_country_holidays(country_name='KR') # 한국의 휴일 날짜를 자동으로 입력해줌
m.fit(series)

future = m.make_future_dataframe(periods = 34)
forecast = m.predict(future)

fig1 = m.plot(forecast)
plt.title('Prophet으로 예측된 값(푸른선) + 실제 값(검은점)')
fig2 = m.plot_components(forecast) # Seasonal Decomposition
plt.show()
# 약간씩 벗어나는 점도 있고, 굉장히 변동이 큰 경우에는 놓치는 점도 있지만, 대체로 잘 맞췄다고 생각해서 이 하이퍼 파라미터 조합으로 실시.

In [ ]:
forecast.loc[:,['ds','yhat']] # ds는 날짜, yhat은 모델이 예측해준 값

- 주기가 7일 , 즉 일주일 마다 반복됨을 알 수 있습니다.
- 이를 참고하여 Seasonal_decompose 로 대분류별로 주기 7에 대한 결과를 추가적으로 확인해봅니다.

In [ ]:
df2 = pd.read_csv('./train.csv')

In [ ]:
df_maincat = df.groupby(by = '대분류').mean().drop(columns = 'ID') #대분류별로 판매량의 평균을 구하고 ID는 더한 값이 있으면 안되니까 제거
df_maincat.head()

In [ ]:
df_maincat_T = df_maincat.transpose()
df_maincat_T.head()

In [ ]:
df_maincat_T_1 = df_maincat_T.loc['2022-01-01':'2023-02-22']

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
df_maincat_T_1.index = pd.to_datetime(df_maincat_T_1.index)

In [ ]:
for i in df_maincat_T_1.columns:
  ts = df_maincat_T_1[i]
  period = 7 # 주별 주기확인
  result = seasonal_decompose(ts, period=period)
  plt.rcParams['figure.figsize'] = [20, 8]
  result.plot()
  plt.xlabel('Date')
  plt.show()

- 추후 모델에 Window size 를 7 단위로 적용시키면 될것 같다.

### 모든 제품에 같은 파라미터를 적용해서 예측

In [ ]:
zero.info() # train의 판매량은 int타입임. 모델로 예측한 값은 float 타입이니까 바로는 안들어간다.

In [ ]:
ds = zero.loc[:,'2022-01-01':'2023-02-22'].columns

In [ ]:
i = 7 # 7번 제품에 대해서 해보기
series = pd.DataFrame({
    'ds':ds,
    'y':zero.loc[i,'2022-01-01':'2023-02-22']
})

In [ ]:
series.head()

In [ ]:
m = Prophet(
    # trend
    changepoint_prior_scale=0.5, # 경향을 더 유연하게 받아들임.
    # seasonality
    seasonality_prior_scale = 0.5, # 계절성도 더 유연하게 받아들임.
    weekly_seasonality = 20, # 주 계절성을 더 민감하게.
    daily_seasonality=False,
    yearly_seasonality=False,
    seasonality_mode='multiplicative' # 계절성의 크기를 가변형으로 잡고
)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5) # monthly 계절성 추가
m.add_country_holidays(country_name='KR') # 한국의 휴일 날짜를 자동으로 입력해줌
# 휴일이란, 해당 날짜의 변동에만 특별한 가중치를 부여하여 이후 진행될 fitting에 영향력을 극단적으로 줄이는 것.
m.fit(series)

future = m.make_future_dataframe(periods = 34)
forecast = m.predict(future)

fig1 = m.plot(forecast)
plt.title('Prophet으로 예측된 값(푸른선) + 실제 값(검은점)')
fig2 = m.plot_components(forecast)

In [ ]:
m = Prophet(
    # trend
    changepoint_prior_scale=0.5, # 경향을 더 유연하게 받아들임.
    # seasonality
    seasonality_prior_scale = 0.5, # 계절성도 더 유연하게 받아들임.
    weekly_seasonality = 20,
    daily_seasonality=False,
    yearly_seasonality=False,
    seasonality_mode='multiplicative' # 계절성의 크기를 가변형으로 잡고
)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5) # monthly 계절성 추가
m.add_country_holidays(country_name='KR') # 한국의 휴일 날짜를 자동으로 입력해줌
m.fit(series)

future = m.make_future_dataframe(periods = 34)
forecast = m.predict(future)

fig1 = m.plot(forecast)
plt.title('Prophet으로 예측된 값(푸른선) + 실제 값(검은점)')
fig2 = m.plot_components(forecast)

In [ ]:
yhat = forecast.loc[forecast['ds'] >= '2023-02-23','yhat'] # prophet으로 예측한 값중 우리가 필요한 날짜만큼 슬라이싱
yhat = yhat.astype(int);yhat

In [ ]:
# zero = train.iloc[((train.loc[:,'2023-02-23':'2023-03-28'] != 0).sum(1) == 0).values,:] # 이건 전체 train 중에서 02/23~03/28 에 집계가 안된 것들
# not_zero = train.iloc[((train.loc[:,'2023-02-23':'2023-03-28'] != 0).sum(1) != 0).values,:] # 이건 전체 train 중에서 02/23~03/28 에 집계가 제대로 된 것들
# print(zero.shape, not_zero.shape, zero.shape[0] + not_zero.shape[0], '정확하게 나눠진 모습')
ds = zero.loc[:,'2022-01-01':'2023-02-22'].columns
ds

### 반복문으로 14722개 제품에 대해서 각각 예측해보기

In [ ]:
'''
from tqdm.auto import tqdm

save_yhat = [] # 각각 제품들의 yhat값을 저장할 빈 리스트 선언

for i in tqdm(zero.index):
  series = pd.DataFrame({
    'ds':ds,
    'y':zero.loc[i,'2022-01-01':'2023-02-22']
  })
  m = Prophet(
    # trend
    changepoint_prior_scale=0.5, # 경향을 더 유연하게 받아들임.
    # seasonality
    seasonality_prior_scale = 0.5,
    weekly_seasonality = 20,
    daily_seasonality=False, # 일계절성 없음
    yearly_seasonality=False, # 연계절성 없음
    seasonality_mode='multiplicative' # 계절성의 크기를 가변형으로 잡고
  )
  m.add_seasonality(name='monthly', period=30.5, fourier_order=5) # monthly 계절성 추가
  m.add_country_holidays(country_name='KR') # 한국의 휴일 날짜를 자동으로 입력해줌. 혹시나 해서 넣어봤다.
  m.fit(series)

  # future = m.make_future_dataframe(periods = 34) # 이건 이전에 실행해두었다면 다시 할 필요없음.
  # 매반복마다 future를 생성하면 오히려 더 늦어질 것.
  forecast = m.predict(future)

  save_yhat.append(forecast.loc[forecast['ds'] >= '2023-02-23','yhat'].values)
  # 리스트의 각 인덱스마다 prophet모형이 418일간 데이터로 예측한 14772개 각 제품에 대한 미래 2023-02-23~2023-03-28까지 판매량을 저장
  # 코렙 무료버전 기준 4시간 20분 가량 소요.
  # 그래서 해당 코드로 만들어낸 CSV파일을 같이 첨부하겠습니다. 구동되는건 확인하였습니다.
  '''

In [ ]:
# pp = pd.DataFrame(save_yhat)

In [ ]:
# pp.to_csv('프로펫으로 예측한 결측치 + holiday.csv',encoding='utf-8-sig',index = False)
# 기존에는 이 내용을 csv로 저장해뒀다가 나중에 불러와서 사용했습니다.
# 반복문 돌리는데 시간이 너무 오래걸려서, 다시하면 시간낭비가 심하니 그걸 방지하기 위해서 CSV로 저장했습니다.

# 단순히 반복문을 다시 돌리는 불상사를 예방하기 위해서 개인적으로 쓰려고 했던 CSV라서 prophet_holiday_replaced_train.csv를 만들고 삭제해서 지금은 가지고 있지 않으니 따로 CSV 파일은 첨부하지 않겠습니다.

### 구한 내용 적용한 CSV 만들기


#####pp = pd.read_csv('프로펫으로 예측한 결측치 + holiday.csv')
###### round(pp).astype(int) # 이건 반올림한 후에 int 타입으로 바꾼 것
###### 처음엔 반올림을 적용해서 csv를 만들겠다.
###### a = round(pp).astype(int).values
###### no_0_val = np.where(a < 0, 0, a) # 이건 배열에서 0보다 작은 값은 전부 0으로 치환해주는 함수
###### 음수라는 건 전부 0일 가능성이 그만큼 더 크다는 뜻이 아닐까? 라는 생각으로 모두 0으로 치환하였습니다.

##### zero.loc[:,'2023-02-23':'2023-03-28'] = no_0_val
##### zero.loc[:,'2023-02-23':'2023-03-28']
##### zero라는 변수는 34일간 0인 결측을 가진 데이터 프레임입니다. 거기에 음수를 모두 0으로 치환한 값을 넣습니다.
##### (zero.loc[:,'2023-02-23':'2023-03-28'] != no_0_val).sum().sum() # 0이면 zero 데이터프레임에 no_0_val이 잘 들어갔다는 의미
---

##### for_csv = pd.concat([zero,not_zero], axis = 0)
##### zero(0이 있는 제품들을 prophet으로 예측한 값을 반올림해서 정수화한 걸 넣은 것) + not_zero(0이 없던 원래 그대로)
##### for_csv = for_csv.sort_values('ID', ascending = True) # ID기준으로 오름차순 정렬

##### (for_csv.isnull().sum() != 0).sum()
##### 데이터 프레임 전체에 결측치 없음.

##### for_csv.groupby('대분류').mean().drop(columns = 'ID').transpose().plot();
##### 그래프 모양을 보면 새롭게 예측한 값이 잘 들어간 것으로 보인다.
##### plt.scatter((for_csv.select_dtypes('int') < 0).sum(1).index, (for_csv.select_dtypes('int') < 0).sum(1).values, s = 5)
##### 새로 만든 데이터프레임에 0보다 작은 값이 있는지 확인. 하나도 없었다. 그럼 잘 된것.

In [ ]:
# for_csv.to_csv('prophet_holiday_replaced_train.csv',encoding='utf-8-sig',index = False) # csv 파일로 내보내기

## prophet_holiday_replaced_train.csv
* 위 제목과 같은 이름의 (2023-02-23 ~ 2023-03-28) 34일간 판매량이 연속으로 0이었던 결측으로 판단하여 Prophet 모형을 이용해 결측의 보간한 train.csv를 앞으로의 학습에 이용하였습니다.
* 아래에 해당 csv 파일의 링크를 첨부하겠습니다. 링크가 있는 모든 사용자는 편집 가능하게 해두었습니다.

https://drive.google.com/file/d/1H3lV12K0K75dqyh2Fc5M9cIW9c-xkPQ5/view?usp=sharing   
!!! 코드와 같이 첨부된 csv 파일과 완전히 동일한 CSV 파일의 다운로드 링크입니다  필요하지 않다면 굳이 열지 않으셔도 됩니다. !!!

# 4. 데이터 Smoothing - Savitzky-Golay filter 사용
* smoothing을 하면 LSTM의 정확도가 올라간다는 내용의 논문에서 아이디어를 얻어서 스무딩을 진행


## prophet_hoilday_replaced_train.csv 파일 불러오기

In [ ]:
train_data = pd.read_csv('./prophet_holiday_replaced_train.csv').drop(columns=['ID', '제품'])
# 위의 해당 csv파일의 링크 첨부해두었음.

In [ ]:
train_data

In [ ]:
_train_data = train_data.iloc[:,4:] # Savitzky-Golay filter를 넣기 위해 대,중,소 분류 및 브랜드 col을 없앤 _train_data를 만든다.
_train_col = _train_data.columns.tolist() # 그리고 날짜 col을 기록하기 위해 list에 담아둔다.
_train_data

In [ ]:
_train_data = signal.savgol_filter(_train_data, window_length = 7, polyorder = 3, mode="nearest")
#_train_data를 savgol_filter에 넣는다. 이때 파라미터는 주기가 7일로 전처리에서 확인하였으므로 window_length는 7일
# polyorder 차수는 적당히 복잡하지 않은 3차수로 하고 mode="nearest"를 사용하여 가장 가까운 데이터 포인트를 사용하여 스무딩을 진행.

In [ ]:
_train_data

In [ ]:
_train_data = pd.DataFrame(_train_data, columns=_train_col)#Savitzky-Golay filter를 넣어 평활한 _train_data를 다시 DataFrame으로 만든다.
_train_data

In [ ]:
train_data = pd.concat([train_data.iloc[:,:4],_train_data], axis=1)#앞에 없앴던 대,중,소분류 및 브랜드 컬럼을 합친 train_data DataFrame으로 만든다.
train_data

# ---------------------------------------------------------------------------------------------------------------------------------------------

#5. 데이터 불러오기

## 프로펫 결측치(KR 휴일 포함) + savgol smoothing + 카테고리.csv
* 위 제목과 같은 이름의 결측치를 보완한 train.csv를 Savitzky-Golay filter를 거쳐서 나온 train.csv를 앞으로의 학습에 이용하였습니다.
* 아래에 해당 csv 파일의 링크를 첨부하겠습니다. 링크가 있는 모든 사용자는 편집 가능하게 해두었습니다.

https://drive.google.com/file/d/1jl2ZVb0EMHAdN-744JuondDZiwmP5uIL/view?usp=sharing   
!!! 코드와 같이 첨부된 csv 파일과 완전히 동일한 CSV 파일의 다운로드 링크입니다  필요하지 않다면 굳이 열지 않으셔도 됩니다. !!!


In [ ]:
train_data = pd.read_csv('./프로펫 결측치(KR 휴일 포함) + savgol smoothing + 카테고리.csv').drop(columns=['ID', '제품'])
train_data

#6. 데이터 전처리

## Hyperparameter Setting_default

In [ ]:
CFG = {
    'TRAIN_WINDOW_SIZE':91, # 91일치로 학습 LSTM에서 튜닝한 결과 91이 압도적으로 성능이 좋게 나옴
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10, # 전체 데이터셋을 몇 번 반복해서 훈련할지를 나타냅니다. 즉, 10번의 반복 학습이 이루어집니다.
    'LEARNING_RATE':55e-6,
    'BATCH_SIZE':4096,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) # 파이썬 해시 함수를 사용할 때 사용되는 시드 값을 설정합니다. 이를 통해 해시 기반의 무작위성을 관리합니다.
    np.random.seed(seed) # NumPy 라이브러리의 무작위 숫자 생성기에 시드를 설정
    torch.manual_seed(seed) #PyTorch 라이브러리의 무작위 숫자 생성기에 시드를 설정합니다.
    torch.cuda.manual_seed(seed) #GPU를 사용하는 경우, 해당 코드를 통해 CUDA 무작위 숫자 생성기에 시드를 설정합니다.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Scaling 및 Labeling

In [ ]:
# Data Scaling 0과 1사이의 값으로 변경
scale_max_dict = {} # 이 딕셔너리들은 스케일링 된 데이터에서 원본 데이터로 되돌리기 위해 최대값과 최소값을 저장하는 역할을 합니다.
scale_min_dict = {}

for idx in tqdm(range(len(train_data))):
    maxi = np.max(train_data.iloc[idx,4:]) # 각 행의 4번째 열부터 마지막 열까지의 데이터에서 최대값과 최소값을 계산합니다.
    mini = np.min(train_data.iloc[idx,4:])

    if maxi == mini : # 최대값과 최소값이 같을 경우, 해당 행의 4번째 열부터 마지막 열까지의 값을 모두 0으로 변환합니다.
        train_data.iloc[idx,4:] = 0 # 이는 모든 값이 같은 경우 스케일링이 의미가 없기 때문입니다.
    else:
        train_data.iloc[idx,4:] = (train_data.iloc[idx,4:] - mini) / (maxi - mini) # 최대값과 최소값이 다른 경우
    #, 해당 행의 4번째 열부터 마지막 열까지의 값을 (원래 값 - 최소값) / (최대값 - 최소값)으로 변환하여 0과 1 사이의 값으로 스케일링합니다.

    scale_max_dict[idx] = maxi # 스케일링 된 데이터의 해당 행에 대한 최대값과 최소값을 딕셔너리에 저장합니다.
    scale_min_dict[idx] = mini # 이렇게 하면 스케일링 된 데이터를 다시 원래의 범위로 되돌릴 수 있습니다.

  0%|          | 0/15890 [00:00<?, ?it/s]

In [ ]:
train_data

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,...,0.476852,0.416667,0.402778,0.254630,0.152778,0.064815,0.111111,0.111111,0.111111,0.111111
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0.047619,0.047619,0.047619,0.047619,0.047619,0.047619,...,0.255952,0.190476,0.202381,0.261905,0.285714,0.220238,0.196429,0.119048,0.107143,0.119048
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0.038186,0.038186,0.038186,0.038186,0.038186,0.038186,...,0.038186,0.038186,0.038186,0.038186,0.038186,0.038186,0.038186,0.038186,0.038186,0.038186
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0.022923,0.022923,0.022923,0.022923,0.022923,0.022923,...,0.022923,0.022923,0.022923,0.022923,0.022923,0.022923,0.022923,0.022923,0.022923,0.022923
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0.073876,0.073876,0.073876,0.073876,0.073876,0.073876,...,0.073876,0.073876,0.073876,0.073876,0.073876,0.073876,0.073876,0.073876,0.073876,0.073876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,0.091150,0.091150,0.091150,0.091150,0.091150,0.091150,...,0.238648,0.183626,0.123301,0.081869,0.084521,0.091150,0.091150,0.091150,0.091150,0.091150
15886,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0.002704,0.002704,0.002704,0.002704,0.002704,0.002704,...,0.391167,0.302839,0.163587,0.073006,0.024335,0.004507,0.019378,0.023434,0.021631,0.022082
15887,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0.011091,0.011091,0.011091,0.011091,0.011091,0.011091,...,0.011091,0.011091,0.011091,0.011091,0.011091,0.011091,0.011091,0.011091,0.011091,0.011091
15888,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0.021898,0.021898,0.021898,0.021898,0.021898,0.021898,...,0.105839,0.094891,0.072993,0.047445,0.025547,0.014599,0.007299,0.029197,0.072993,0.124088


In [ ]:
# Label Encoding
label_encoder = LabelEncoder() # 먼저 LabelEncoder 객체를 생성합니다. 이 객체는 카테고리형 데이터를 숫자로 인코딩하는데 사용됩니다.
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns: # categorical_columns 리스트에 있는 각 카테고리형 변수들을 순회하면서 라벨 인코딩을 수행합니다.
    label_encoder.fit(train_data[col]) #변수에 나타나는 고유한 카테고리들을 파악하여 각 카테고리에 대해 고유한 숫자를 매핑할 수 있도록 준비(학습)합니다.
    train_data[col] = label_encoder.transform(train_data[col]) # 학습된 LabelEncoder 객체를 사용하여 해당 카테고리형 변수의 값들을 숫자로 변환합니다.
    #각 카테고리별로 고유한 숫자로 매핑하여 데이터를 변환합니다.

## Dataset 생성 및 Data split


In [ ]:
# 해당 함수는 일별 판매량 데이터를 입력으로 받아서 학습 기간 블럭과 예측 기간 블럭의 세트로 데이터를 생성하는 기능을 수행합니다.
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간 = 90일
    predict_size : 추론할 기간 = 21
    '''
    num_rows = len(data) # 데이터프레임의 행(row) 개수를 num_rows에 저장합니다.
    window_size = train_size + predict_size # 학습 기간 블럭과 예측 기간 블럭의 크기를 합친 값을 window_size에 저장합니다.

    # 학습용 입력 데이터를 저장할 input_data 배열과 예측 결과를 저장할 target_data 배열을 빈 배열로 초기화합니다.
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4]) # 각 행의 '대분류', '중분류', '소분류', '브랜드'에 해당하는 값을 encode_info로 추출합니다.
        sales_data = np.array(data.iloc[i, 4:]) # 각 행의 판매량 데이터를 sales_data로 추출합니다.

        for j in range(len(sales_data) - window_size + 1): # 각 행의 판매량 데이터를 순회하면서 학습용 입력 데이터(input_data)와 예측 결과(target_data)를 생성합니다.
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            # 학습 기간 블럭 크기만큼의 판매량 데이터를 담는 temp_data를 생성합니다.
            # 이때, 해당 행의 제품 정보인 encode_info를 반복하여 모든 데이터에 추가하고, 학습 기간 블럭 크기만큼의 판매량 데이터를 temp_data와 함께 열 방향으로 결합합니다.
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]

    return input_data, target_data

In [ ]:
# 해당 함수는 일별 판매량 데이터를 입력으로 받아서 추론에 사용할 입력 데이터를 생성하는 기능을 수행합니다.
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간) = 90일
    '''
    num_rows = len(data) # 데이터프레임의 행(row) 개수를 num_rows에 저장합니다.

    #추론에 사용할 입력 데이터를 저장할 input_data 배열을 빈 배열로 초기화합니다.
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4]) # 각 행의 '대분류', '중분류', '소분류', '브랜드'에 해당하는 값을 encode_info로 추출합니다.
        sales_data = np.array(data.iloc[i, -train_size:])

        # 학습 기간 크기만큼의 판매량 데이터를 담는 temp_data를 생성합니다.
        # 이때, 해당 행의 제품 정보인 encode_info를 반복하여 모든 데이터에 추가하고, 학습 기간 크기만큼의 판매량 데이터를 temp_data와 함께 열 방향으로 결합합니다.
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data

    return input_data

In [ ]:
# 위 코드는 make_train_data 함수와 make_predict_data 함수를 사용하여 학습 데이터와 테스트 데이터를 생성
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [ ]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [ ]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4474624, 91, 5),
 (4474624, 21),
 (1118656, 91, 5),
 (1118656, 21),
 (15890, 91, 5))

## Custom Dataset

In [ ]:
# 위 코드는 PyTorch에서 사용하는 커스텀 데이터셋(CustomDataset) 클래스를 정의하는 부분입니다.
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index): #메서드는 데이터셋에서 특정 인덱스의 데이터를 가져올 때 호출됩니다.
        if self.Y is not None: # self.Y가 None이 아니면, 입력 데이터와 타겟 데이터를 함께 반환합니다.
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index]) # 인덱스를 통해 self.X와 self.Y에서 데이터를 가져옵니다.
        return torch.Tensor(self.X[index])

    def __len__(self):
        return len(self.X) # len(self.X)를 호출하여 데이터셋의 샘플 개수를 반환합니다.

In [ ]:
# CustomDataset 클래스를 활용하여 학습용 데이터셋과 검증용 데이터셋을 생성하고,
# DataLoader를 사용하여 배치 단위로 데이터를 불러오는 부분입니다.

train_dataset = CustomDataset(train_input, train_target) # CustomDataset 클래스를 사용하여 train_dataset을 생성합니다.
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
# DataLoader는 batch_size에 지정된 배치 크기만큼 데이터를 한 번에 로드합니다.shuffle=True로 설정되어 있으므로 데이터를 섞어서 로드합니다. 이는 모델의 학습이 더 잘 이루어지도록 합니다.

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

#7. 모델 선언(Bi-LSTM)

In [ ]:
class biLSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length, drop_out):
        super(biLSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.drop_out = drop_out

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, bidirectional=True, batch_first=True)

        self.fc =  nn.Sequential(
            nn.Linear(hidden_size*2, hidden_size),
            nn.ReLU(),
            nn.Dropout(p = drop_out),
            nn.Linear(hidden_size, num_classes)
        )
        self.relu = nn.ReLU()

    def forward(self,x):
        h_0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size, device=device)
        c_0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size, device=device)
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        return out

#8. 모델 튜닝(Optuna)

In [ ]:
# 위 함수는 머신 러닝 모델을 튜닝시키는 기능을 수행합니다.
def train_for_tune(model, optimizer, train_loader, val_loader, device): # train_loader = 데이터셋에서 배치 크기만큼 데이터를 로드하는 역할
    model.to(device) # GPU가 지정되었다면 모델을 GPU에 할당하여 가속화된 계산을 수행합니다.
    criterion = nn.MSELoss().to(device) # 평균 제곱 오차(Mean Squared Error, MSE)를 손실 함수로 사용합니다.
    best_loss = 9999999 # 최고 손실 값을 초기화합니다.
    best_model = None # 최고 성능을 보이는 모델을 저장하는 변수를 초기화

    for epoch in range(1, CFG['EPOCHS']+1): # epoch 수만큼 모델을 훈련
        model.train() # 모델을 학습 모드로 설정
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)): # train_loader로부터 미니배치를 가져와서 모델을 학습
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad() #optimizer.zero_grad()로 기울기를 초기화

            output = model(X) #모델을 통해 입력 데이터(X)를 전달하여 예측 결과(output)를 얻습니다.
            loss = criterion(output, Y) # 손실 함수(criterion)를 사용하여 예측 결과와 실제 값 사이의 손실을 계산합니다.

            loss.backward() # 손실(loss)를 역전파하여 기울기를 계산하고,
            optimizer.step() # optimizer.step()으로 가중치를 업데이트합니다.

            train_loss.append(loss.item()) # 학습된 손실 값들을 train_loss 리스트에 저장합니다.

        val_loss = validation(model, val_loader, criterion, device) # validation 함수를 호출하여 검증 데이터셋에 대한 손실 값을 얻어옵니다.
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        # 현재 손실 값이 이전까지의 최고 손실 값보다 작은 경우에만 모델을 저장합니다.
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_loss

In [ ]:
# 위 함수는 학습된 모델을 검증(평가)하기 위한 기능을 수행합니다.
def validation(model, val_loader, criterion, device):
    model.eval() # 모델을 평가 모드로 설정합니다.
    val_loss = []

    # 검증 데이터에 대한 연산에서는 기울기 계산이 필요 없으므로 torch.no_grad() 컨텍스트 매니저를 사용합니다.
    # 이렇게 하면 연산 속도가 향상되며 메모리를 더 효율적으로 사용할 수 있습니다.
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            loss = criterion(output, Y)

            val_loss.append(loss.item())
    return np.mean(val_loss)
    # 모든 미니배치에 대한 검증 데이터의 손실 값을 평균하여 반환합니다.
    # 작은 값일수록 모델의 성능이 더 좋다고 판단할 수 있습니다.

In [ ]:
# 하이퍼파라미터 튜닝 학습모델 초기화
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='minimize')

[I 2023-08-26 09:28:34,315] A new study created in memory with name: no-name-b4d14183-11b4-4241-b857-d0eec6a9b137


In [ ]:
# 위 함수는 하이퍼파라미터 튜닝을 위해 목적함수를 정하는 함수입니다.
def objective(trial):

    global device
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    CFG = {
        'TRAIN_WINDOW_SIZE': 91, #trial.suggest_categorical('TRAIN_WINDOW_SIZE', [m*7 for m in range(1,13+1)]),
        'PREDICT_SIZE':21, # 21일치 예측
        'EPOCHS':10, #trial.suggest_categorical('EPOCHS', [5, 10, 20]),
        'LEARNING_RATE':trial.suggest_loguniform('LEARNING_RATE',1e-5, 1e-4), #1e-5, 1e-3
        'BATCH_SIZE': 4096, #trial.suggest_categorical('BATCH_SIZE', [4096, 8192]),
        'DROP_OUT': trial.suggest_loguniform('DROP_OUT', 0.1, 0.3),
        'SEED':41
    }

    model = biLSTM(num_classes= CFG['PREDICT_SIZE'], input_size= 5, hidden_size = 512, num_layers =2,
                   seq_length= CFG['PREDICT_SIZE'], drop_out = CFG['DROP_OUT'])
    model = nn.DataParallel(model,device_ids = [0,1])
    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    val_loss = train_for_tune(model, optimizer, train_loader, val_loader, device= device)

    return val_loss


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print(study.best_trial.params)

[I 2023-08-26 09:32:38,568] A new study created in memory with name: no-name-2fe05d68-7ab9-4cb7-9b03-3561e19e856d
/tmp/ipykernel_3504829/2331593271.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'LEARNING_RATE':trial.suggest_loguniform('LEARNING_RATE',1e-5, 1e-4), #1e-5, 1e-3
/tmp/ipykernel_3504829/2331593271.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'DROP_OUT': trial.suggest_loguniform('DROP_OUT', 0.1, 0.3),


  0%|          | 0/1093 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.03038] Val Loss : [0.02811]
Model Saved


  0%|          | 0/1093 [00:00<?, ?it/s]

...

Epoch : [9] Train Loss : [0.01647] Val Loss : [0.01640]
Model Saved


  0%|          | 0/1093 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

[I 2023-08-26 20:28:35,946] Trial 9 finished with value: 0.01630544997297098 and parameters: {'LEARNING_RATE': 2.762700587248211e-05, 'DROP_OUT': 0.1367532107572949}. Best is trial 7 with value: 0.015003435929768114.


Epoch : [10] Train Loss : [0.01633] Val Loss : [0.01631]
Model Saved
{'LEARNING_RATE': 9.898609422754722e-05, 'DROP_OUT': 0.110295193300282}


In [ ]:
# 하이퍼파라미터 튜닝 결과 저장
joblib.dump(study, '/shared/home/inhamot/LG_AI/data/biLSTM_optuna.pkl')

['/shared/home/inhamot/LG_AI/data/biLSTM_optuna.pkl']

In [ ]:
study = joblib.load('/shared/home/inhamot/LG_AI/data/biLSTM_optuna.pkl')
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_DROP_OUT,params_LEARNING_RATE,state
0,0,0.017140,2023-08-26 09:32:38.570984,2023-08-26 10:38:16.027655,0 days 01:05:37.456671,0.234785,0.000019,COMPLETE
1,1,0.016170,2023-08-26 10:38:16.029031,2023-08-26 11:43:44.630999,0 days 01:05:28.601968,0.197214,0.000020,COMPLETE
2,2,0.015364,2023-08-26 11:43:44.632166,2023-08-26 12:49:15.364257,0 days 01:05:30.732091,0.157676,0.000049,COMPLETE
3,3,0.017311,2023-08-26 12:49:15.365402,2023-08-26 13:54:49.501222,0 days 01:05:34.135820,0.245685,0.000017,COMPLETE
4,4,0.016451,2023-08-26 13:54:49.502429,2023-08-26 15:00:27.080567,0 days 01:05:37.578138,0.206400,0.000024,COMPLETE
5,5,0.016538,2023-08-26 15:00:27.081746,2023-08-26 16:06:07.617400,0 days 01:05:40.535654,0.240683,0.000019,COMPLETE
6,6,0.015237,2023-08-26 16:06:07.618625,2023-08-26 17:11:55.046875,0 days 01:05:47.428250,0.109286,0.000048,COMPLETE
7,7,0.015003,2023-08-26 17:11:55.047908,2023-08-26 18:17:28.652202,0 days 01:05:33.604294,0.110295,0.000099,COMPLETE
8,8,0.016076,2023-08-26 18:17:28.653371,2023-08-26 19:23:02.572823,0 days 01:05:33.919452,0.206611,0.000050,COMPLETE
9,9,0.016305,2023-08-26 19:23:02.574071,2023-08-26 20:28:35.946602,0 days 01:05:33.372531,0.136753,0.000028,COMPLETE


In [ ]:
df.sort_values(by=['value'])

,number,value,datetime_start,datetime_complete,duration,params_DROP_OUT,params_LEARNING_RATE,state
7,7,0.015003,2023-08-26 17:11:55.047908,2023-08-26 18:17:28.652202,0 days 01:05:33.604294,0.110295,0.000099,COMPLETE
6,6,0.015237,2023-08-26 16:06:07.618625,2023-08-26 17:11:55.046875,0 days 01:05:47.428250,0.109286,0.000048,COMPLETE
2,2,0.015364,2023-08-26 11:43:44.632166,2023-08-26 12:49:15.364257,0 days 01:05:30.732091,0.157676,0.000049,COMPLETE
8,8,0.016076,2023-08-26 18:17:28.653371,2023-08-26 19:23:02.572823,0 days 01:05:33.919452,0.206611,0.000050,COMPLETE
1,1,0.016170,2023-08-26 10:38:16.029031,2023-08-26 11:43:44.630999,0 days 01:05:28.601968,0.197214,0.000020,COMPLETE
9,9,0.016305,2023-08-26 19:23:02.574071,2023-08-26 20:28:35.946602,0 days 01:05:33.372531,0.136753,0.000028,COMPLETE
4,4,0.016451,2023-08-26 13:54:49.502429,2023-08-26 15:00:27.080567,0 days 01:05:37.578138,0.206400,0.000024,COMPLETE
5,5,0.016538,2023-08-26 15:00:27.081746,2023-08-26 16:06:07.617400,0 days 01:05:40.535654,0.240683,0.000019,COMPLETE
0,0,0.017140,2023-08-26 09:32:38.570984,2023-08-26 10:38:16.027655,0 days 01:05:37.456671,0.234785,0.000019,COMPLETE
3,3,0.017311,2023-08-26 12:49:15.365402,2023-08-26 13:54:49.501222,0 days 01:05:34.135820,0.245685,0.000017,COMPLETE


#9. 모델 학습

In [ ]:
# 튜닝하여 최적화된 하이퍼파라미터
CFG = {
    'TRAIN_WINDOW_SIZE':91, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10, # 전체 데이터셋을 몇 번 반복해서 훈련할지를 나타냅니다. 즉, 10번의 반복 학습이 이루어집니다.
    'LEARNING_RATE':9.898609422754722e-05,
    'BATCH_SIZE':4096,
    'DROP_OUT':0.110295193300282,
    'SEED':41
}

In [ ]:
# 위 함수는 머신 러닝 모델을 훈련시키는 기능을 수행합니다.
def train(model, optimizer, train_loader, val_loader, device): # train_loader = 데이터셋에서 배치 크기만큼 데이터를 로드하는 역할
    model.to(device) # GPU가 지정되었다면 모델을 GPU에 할당하여 가속화된 계산을 수행합니다.
    criterion = nn.MSELoss().to(device) # 평균 제곱 오차(Mean Squared Error, MSE)를 손실 함수로 사용합니다.
    best_loss = 9999999 # 최고 손실 값을 초기화합니다.
    best_model = None # 최고 성능을 보이는 모델을 저장하는 변수를 초기화

    for epoch in range(1, CFG['EPOCHS']+1): # epoch 수만큼 모델을 훈련
        model.train() # 모델을 학습 모드로 설정
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)): # train_loader로부터 미니배치를 가져와서 모델을 학습
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad() #optimizer.zero_grad()로 기울기를 초기화

            output = model(X) #모델을 통해 입력 데이터(X)를 전달하여 예측 결과(output)를 얻습니다.
            loss = criterion(output, Y) # 손실 함수(criterion)를 사용하여 예측 결과와 실제 값 사이의 손실을 계산합니다.

            loss.backward() # 손실(loss)를 역전파하여 기울기를 계산하고,
            optimizer.step() # optimizer.step()으로 가중치를 업데이트합니다.

            train_loss.append(loss.item()) # 학습된 손실 값들을 train_loss 리스트에 저장합니다.

        val_loss = validation(model, val_loader, criterion, device) # validation 함수를 호출하여 검증 데이터셋에 대한 손실 값을 얻어옵니다.
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        # 현재 손실 값이 이전까지의 최고 손실 값보다 작은 경우에만 모델을 저장합니다.
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

In [ ]:
model = biLSTM(num_classes= CFG['PREDICT_SIZE'], input_size= 5, hidden_size = 512, num_layers =2,
               seq_length= CFG['PREDICT_SIZE'], drop_out =CFG['DROP_OUT']
model = nn.DataParallel(model,device_ids = [0,1]) ## 모델에 GPU 2개 병렬 연결
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device= device)

  0%|          | 0/1093 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.03089] Val Loss : [0.03053]
Model Saved


  0%|          | 0/1093 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

...

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch : [9] Train Loss : [0.01479] Val Loss : [0.01464]
Model Saved


  0%|          | 0/1093 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch : [10] Train Loss : [0.01462] Val Loss : [0.01478]


#10. 예측결과 도출

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)

            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()

            predictions.extend(output)

    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader, device)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

(15890, 21)

In [ ]:
pred

array([[2, 3, 3, ..., 4, 4, 4],
       [1, 2, 2, ..., 2, 2, 2],
       [3, 3, 3, ..., 2, 2, 2],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,2,3,3,3,3,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
1,1,1,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,3,3,3,3,2,3,3,3,3,...,2,2,2,2,2,2,2,2,2,2
3,3,5,5,5,5,4,4,4,4,4,...,3,4,4,4,4,3,3,3,3,3
4,4,3,3,3,3,3,4,4,4,3,...,4,4,4,4,4,4,4,4,4,4


In [ ]:
submit.to_csv('./BiLSTM_best_param_submit.csv', index=False)